In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# EDA
import statsmodels.api as sts
from pylab import rcParams

# ADF
from statsmodels.tsa.stattools import adfuller

# ACF & PACF
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# AR, MA and ARIMA
from statsmodels.tsa.arima_model import ARIMA

# Scale 
from fbprophet import Prophet

In [36]:
df= pd.read_csv('household_power_consumption_1.txt',sep=';',
                         parse_dates={'dt' : ['Date', 'Time']}, infer_datetime_format = True,
                         low_memory=False, na_values=['nan','?'],index_col='dt')

df.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0
2006-12-16 17:27:00,5.388,0.502,233.74,23.0,0.0,1.0,17.0
2006-12-16 17:28:00,3.666,0.528,235.68,15.8,0.0,1.0,17.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2075259 entries, 2006-12-16 17:24:00 to 2010-11-26 21:02:00
Data columns (total 7 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Global_active_power    float64
 1   Global_reactive_power  float64
 2   Voltage                float64
 3   Global_intensity       float64
 4   Sub_metering_1         float64
 5   Sub_metering_2         float64
 6   Sub_metering_3         float64
dtypes: float64(7)
memory usage: 126.7 MB


In [20]:
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace= True)

In [5]:
df.head()

,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Date,,,,,,,,
2006-12-16,17:24:00,4.216,0.418,234.840,18.400,0.000,1.000,17.0
2006-12-16,17:25:00,5.360,0.436,233.630,23.000,0.000,1.000,16.0
2006-12-16,17:26:00,5.374,0.498,233.290,23.000,0.000,2.000,17.0
2006-12-16,17:27:00,5.388,0.502,233.740,23.000,0.000,1.000,17.0
2006-12-16,17:28:00,3.666,0.528,235.680,15.800,0.000,1.000,17.0


In [38]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [39]:
for j in range(0,7):
    df.iloc[:,j] = df.iloc[:,j].fillna(df.iloc[:,j].mean())

In [40]:
from scipy.stats import pearsonr, kendalltau

In [44]:
pearsonr(df['Global_active_power'], df['Global_reactive_power'])

(0.24701705264406365, 0.0)

In [46]:
pearsonr(df['Voltage'], df['Global_intensity'])

(-0.41136307176847076, 0.0)

In [9]:
#df['Global_reactive_power'] = df['Global_reactive_power'].astype(float)
# df['Global_active_power'].apply(lambda x: float(x))

In [54]:
df_daily= df.resample('D').sum()
df_daily.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0
2006-12-17,3390.460,226.006,345725.32,14398.6,2033.0,4187.0,13341.0
2006-12-18,2203.826,161.792,347373.64,9247.2,1063.0,2621.0,14018.0
2006-12-19,1666.194,150.942,348479.01,7094.0,839.0,7602.0,6197.0
2006-12-20,2225.748,160.998,348923.61,9313.0,0.0,2648.0,14063.0


In [49]:
df_ = df[['Global_active_power', 'Global_reactive_power']]
df_.corr(method ='pearson') 

,Global_active_power,Global_reactive_power
Global_active_power,1.000000,0.247017
Global_reactive_power,0.247017,1.000000


In [55]:
df_daily.head()

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
dt,,,,,,,
2006-12-16,1209.176,34.922,93552.53,5180.8,0.0,546.0,4926.0
2006-12-17,3390.460,226.006,345725.32,14398.6,2033.0,4187.0,13341.0
2006-12-18,2203.826,161.792,347373.64,9247.2,1063.0,2621.0,14018.0
2006-12-19,1666.194,150.942,348479.01,7094.0,839.0,7602.0,6197.0
2006-12-20,2225.748,160.998,348923.61,9313.0,0.0,2648.0,14063.0


In [57]:
#x = df_daily.drop('global_active_power')
#y = df_daily['global_active_power']
#values = df_daily.values()
#n_train = 1095
#train = values[:n_train, :]
#test = values[n_train:, :]

In [ ]:
#df= df.rename(columns= {'dt': 'ds', 'global_active_power': 'y'})
#df.head()